In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
!pip install pytorchcv

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import fastai
from fastai.vision import *
from pytorchcv.model_provider import get_model as ptcv_get_model
import warnings
warnings.filterwarnings('ignore')

In [5]:
print(fastai.__version__)

1.0.59


In [6]:
path = Path("/content/drive/My Drive/dla-project/data/")
path.ls()

[PosixPath('/content/drive/My Drive/dla-project/data/test'),
 PosixPath('/content/drive/My Drive/dla-project/data/train'),
 PosixPath('/content/drive/My Drive/dla-project/data/test_single')]

In [7]:
bs = 32 
size1 = 320
size2 = 362
tfms = ([crop_pad()], [])

il = ImageList.from_folder(path/'test')
ils = il.split_none()
ll = ils.label_from_folder()
ll.valid = ll.train
ll.test = ll.train

effnet_ll = ll 
effnet_ll.transform(tfms=tfms,size=size1)
effnet_data = effnet_ll.databunch(bs=bs);
effnet_data.normalize(imagenet_stats)

ImageDataBunch;

Train: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential
Path: /content/drive/My Drive/dla-project/data/test;

Valid: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential
Path: /content/drive/My Drive/dla-project/data/test;

Test: LabelList (49 items)
x: ImageList
Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320),Image (3, 320, 320)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential


In [8]:
incept_ll = ll 
incept_ll.transform(tfms=tfms,size=size2)
incept_data = incept_ll.databunch(bs=bs);
incept_data.normalize(imagenet_stats)

ImageDataBunch;

Train: LabelList (49 items)
x: ImageList
Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential
Path: /content/drive/My Drive/dla-project/data/test;

Valid: LabelList (49 items)
x: ImageList
Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential
Path: /content/drive/My Drive/dla-project/data/test;

Test: LabelList (49 items)
x: ImageList
Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362),Image (3, 362, 362)
y: CategoryList
no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential,no_turbines_high_potential


In [0]:
inference_test_efficientnet = load_learner(path, file=path/'train/effnet-b1-c98.pkl')
inference_test_inceptionv3 = load_learner(path, file=path/'train/incept-v3-c95.pkl')

In [0]:
inference_test_efficientnet.data = effnet_data
inference_test_inceptionv3.data = incept_data

In [11]:
print(fastai.vision.defaults.device)

cuda


In [12]:
%time effnet_results = inference_test_efficientnet.get_preds(ds_type=DatasetType.Test)

CPU times: user 570 ms, sys: 449 ms, total: 1.02 s
Wall time: 14 s


In [13]:
effnet_top_1 = top_k_accuracy(effnet_results[0], effnet_results[1], k=1)
effnet_top_3 = top_k_accuracy(effnet_results[0], effnet_results[1], k=3)
effnet_top_5 = top_k_accuracy(effnet_results[0], effnet_results[1], k=5)

print("Top 1% Accuracy: ", effnet_top_1)
print("Top 3% Accuracy: ", effnet_top_3)
print("Top 5% Accuracy: ", effnet_top_5)

Top 1% Accuracy:  tensor(0.7551)
Top 3% Accuracy:  tensor(0.9796)
Top 5% Accuracy:  tensor(0.9796)


In [14]:
inference_test_efficientnet.show_results(rows=9)

Output hidden; open in https://colab.research.google.com to view.

In [15]:
%time incept_results = inference_test_inceptionv3.get_preds(ds_type=DatasetType.Test)

CPU times: user 608 ms, sys: 473 ms, total: 1.08 s
Wall time: 14.1 s


In [16]:
incept_top_1 = top_k_accuracy(incept_results[0], incept_results[1], k=1)
incept_top_3 = top_k_accuracy(incept_results[0], incept_results[1], k=3)
incept_top_5 = top_k_accuracy(incept_results[0], incept_results[1], k=5)

print("Top 1% Accuracy: ", incept_top_1)
print("Top 3% Accuracy: ", incept_top_3)
print("Top 5% Accuracy: ", incept_top_5)

Top 1% Accuracy:  tensor(0.6735)
Top 3% Accuracy:  tensor(0.9388)
Top 5% Accuracy:  tensor(0.9592)


In [17]:
inference_test_inceptionv3.show_results(rows=9)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
inference_effnet_single = load_learner(path, file=path/'train/effnet-b1-c98.pkl', test=ImageList.from_folder(path/"test_single"))
inference_incept_single = load_learner(path, file=path/'train/incept-v3-c95.pkl', test=ImageList.from_folder(path/"test_single"))


In [19]:
test_files = path/"test_single"
test_name_list = os.listdir(test_files)
test_name_list[:5]

['test_high_capacity_1.jpg',
 'test_high_capacity_2.jpg',
 'test_no_turbines_med_potential_2.jpg',
 'test_high_capacity_3.jpg',
 'test_no_turbines_no_potential_1.jpg']

In [20]:
inference_effnet_single.model.eval()

Sequential(
  (0): Sequential(
    (init_block): EffiInitBlock(
      (conv): ConvBlock(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activ): Swish()
      )
    )
    (stage1): Sequential(
      (unit1): EffiDwsConvUnit(
        (dw_conv): ConvBlock(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activ): Swish()
        )
        (se): SEBlock(
          (pool): AdaptiveAvgPool2d(output_size=1)
          (conv1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (activ): Swish()
          (conv2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (sigmoid): Sigmoid()
        )
        (pw_conv): ConvBlock(
          (conv): Conv2d(32, 16, kernel_size=(1, 

In [21]:
import time
preds = {}
start = time.time()
for i in range(0, len(test_name_list)):
  img_name = str(test_name_list[i])
  img = inference_effnet_single.data.test_dl.dl.dataset.x[i]
  p = inference_effnet_single.predict(img)
  preds[img_name] = str(p[0]); 
end = time.time()
pred_list = list(preds.items())
print("elapsed: ", end - start)

results = pd.DataFrame(pred_list, columns=["file_name", "id"])
print(results)

elapsed:  16.582722902297974
                                file_name                          id
0                test_high_capacity_1.jpg      turbines_high_capacity
1                test_high_capacity_2.jpg      turbines_high_capacity
2    test_no_turbines_med_potential_2.jpg   no_turbines_med_potential
3                test_high_capacity_3.jpg      turbines_high_capacity
4     test_no_turbines_no_potential_1.jpg    no_turbines_no_potential
5                test_high_capacity_4.jpg      turbines_high_capacity
6                test_high_capacity_5.jpg      turbines_high_capacity
7                test_high_capacity_6.jpg      turbines_high_capacity
8                test_high_capacity_7.jpg      turbines_high_capacity
9                test_high_capacity_8.jpg      turbines_high_capacity
10                test_low_capacity_1.jpg   no_turbines_low_potential
11                test_low_capacity_2.jpg       turbines_low_capacity
12                test_low_capacity_3.jpg       turbines_low_

In [22]:
inference_incept_single.model.eval()

Sequential(
  (0): Sequential(
    (init_block): InceptInitBlock(
      (conv1): InceptConv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (activ): ReLU(inplace=True)
      )
      (conv2): InceptConv(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (activ): ReLU(inplace=True)
      )
      (conv3): InceptConv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (activ): ReLU(inplace=True)
      )
      (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (conv4): InceptConv(
        (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [23]:
preds = {}
start = time.time()
for i in range(0, len(test_name_list)):
  img_name = str(test_name_list[i])
  img = inference_incept_single.data.test_dl.dl.dataset.x[i]
  p = inference_incept_single.predict(img)
  preds[img_name] = str(p[0]); 
end = time.time()
pred_list = list(preds.items())
print("elapsed: ", end - start)

results = pd.DataFrame(pred_list, columns=["file_name", "id"])
print(results)

elapsed:  17.835118770599365
                                file_name                          id
0                test_high_capacity_1.jpg    turbines_medium_capacity
1                test_high_capacity_2.jpg      turbines_high_capacity
2    test_no_turbines_med_potential_2.jpg    turbines_medium_capacity
3                test_high_capacity_3.jpg    turbines_medium_capacity
4     test_no_turbines_no_potential_1.jpg    no_turbines_no_potential
5                test_high_capacity_4.jpg      turbines_high_capacity
6                test_high_capacity_5.jpg      turbines_high_capacity
7                test_high_capacity_6.jpg    turbines_medium_capacity
8                test_high_capacity_7.jpg    turbines_medium_capacity
9                test_high_capacity_8.jpg    turbines_medium_capacity
10                test_low_capacity_1.jpg       turbines_low_capacity
11                test_low_capacity_2.jpg       turbines_low_capacity
12                test_low_capacity_3.jpg       turbines_low_